# Testing Individual components of the FV HE scheme

In [1]:
import random

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encryptor import Encryptor
from syft.frameworks.torch.he.fv.decryptor import Decryptor
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType
from syft.frameworks.torch.he.fv.evaluator import Evaluator

## Keygeneration

In [2]:
poly_modulus = 64
bit_sizes= [40]
plain_modulus = 64
ctx = Context(EncryptionParams(poly_modulus, CoeffModulus().create(poly_modulus, bit_sizes), plain_modulus))
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [3]:
print(ctx.param.coeff_modulus)

AttributeError: 'Context' object has no attribute 'param'

In [4]:
# print(len(sk.data))
print('secret key values : ', sk.data)

secret key values :  [[0, 1, 0, 1099511623296, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1099511623296, 1, 0, 1099511623296, 1099511623296, 0, 1099511623296, 0, 1, 0, 1, 1099511623296, 0, 0, 1099511623296, 1099511623296, 1099511623296, 1, 1, 1099511623296, 0, 1, 1, 0, 0, 1099511623296, 1, 1099511623296, 1099511623296, 1099511623296, 1099511623296, 1, 1, 1099511623296, 1, 0, 1099511623296, 1099511623296, 0, 0, 0, 1099511623296, 0, 0, 1099511623296, 1099511623296, 0, 1099511623296, 1, 0]]


In [5]:
# print(pk.data)
# print('public key values : ', pk.data)

## Integer Encoder
Encodes Integer values to Plaintext object

In [6]:
int_encoder = IntegerEncoder(ctx)
ri1 = random.randint(0,10)
ri2 = random.randint(0,10)
ri3 = random.randint(0,10)
pt1 = int_encoder.encode(ri1)
pt2 = int_encoder.encode(ri2)
pt3 = int_encoder.encode(ri3)
print(pt1.data,"   ", pt2.data, "   ", pt3.data)
# print('plaintext data',plaintext.data)

[0, 1]     [0, 0, 0, 1]     [0, 0, 0, 1]


### Decodes back to Integer

In [7]:
print(int_encoder.decode(pt1))
print(int_encoder.decode(pt2))
print(int_encoder.decode(pt3))

2
8
8


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [8]:
encrypter = Encryptor(ctx, pk)

In [9]:
ct1 = encrypter.encrypt(pt1)
ct2 = encrypter.encrypt(pt2)
ct3 = encrypter.encrypt(pt3)

Encrypt Plaintext to ciphertext using secret_key

## Decryptor
Decrypt Ciphertext to Plaintext using secret_key

In [10]:
decrypter = Decryptor(ctx, sk)

In [11]:
dec1 = decrypter.decrypt(ct1)
dec2 = decrypter.decrypt(ct2)
dec3 = decrypter.decrypt(ct3)

In [12]:
print(int_encoder.decode(dec1), "   ", int_encoder.decode(dec2), "   ", int_encoder.decode(dec3))

2     8     8


## Evaluator

In [13]:
eval = Evaluator(ctx)

In [14]:
cc12 = eval.add(ct1, ct2)
cc12 = decrypter.decrypt(cc12)
print(int_encoder.decode(cc12))

10


In [15]:
pc12 = eval.add(pt1, ct2)
pc12 = decrypter.decrypt(pc12)
print(int_encoder.decode(pc12))

10


In [16]:
pp12 = eval.add(pt1, pt2)
print(int_encoder.decode(pp12))

10


### Verify result

In [17]:
assert int_encoder.decode(cc12) == int_encoder.decode(pc12) == int_encoder.decode(pp12) == ri1+ri2

In [18]:
result = eval._mul_cipher_cipher(ct1, ct2)
print("\n\nct1 :",ct1.data)
print("\n\nct2 :",ct2.data)
print('\n\n')

result = decrypter.decrypt(result)
result = int_encoder.decode(result)

print('final result: ', result)



ct1 : [[[152864357394, 314194766596, 660340414334, 730649127007, 508964065424, 1058522846275, 23557869022, 77312731088, 66695944280, 785625586306, 690891491323, 603550672214, 991277537375, 432236537526, 806832477099, 956660206742, 756878992034, 643895437588, 122309769916, 61953303052, 920148784045, 513355255496, 34293603073, 1012443703742, 1033567500594, 994796338203, 970371876072, 640316009828, 1086625859856, 893888601784, 661702282569, 831084835436, 1026757012843, 672213561902, 765373031073, 132438930351, 450070377584, 368904734177, 539497343609, 919712651721, 313048223242, 1060102270531, 839915313921, 58540167032, 565091517923, 605284833752, 1050353437806, 524709766132, 1098265595700, 706970898998, 470639983470, 501577481401, 764107882068, 921545109436, 914730583108, 557306799683, 572668246878, 481428647263, 946700855367, 912384958544, 38950762517, 629176510299, 1093191785838, 540958102478]], [[55716783718, 617570538342, 288497768680, 1030971953362, 344431595013, 681410104196, 640

In [19]:
print(ri1 * ri2, "    ", result)
assert ri1 * ri2 == result

16      16


In [20]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
context = Context(EncryptionParams(128, CoeffModulus().create(128, [30]), 128))

In [21]:
keys = KeyGenerator(context).keygen()

In [22]:
x = th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [23]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12f033f70>

In [24]:
y = x.decrypt(private_key = keys[0])
y

tensor([[1, 2, 3],
        [4, 5, 6]])

In [25]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12f033f70>

In [26]:
x = x + y

In [27]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12f033d90>

In [28]:
x.decrypt(private_key = keys[0])

tensor([[ 2,  4,  6],
        [ 8, 10, 12]])

In [29]:
x = x - th.Tensor([[1,2,3],[4,5,6]]).int().encrypt(protocol="bfv", private_key=keys[0], context = context)

In [30]:
x

(Wrapper)>BFVTensor>[[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x12f063af0>

In [31]:
x.decrypt(private_key = keys[0])

tensor([[1, 2, 3],
        [4, 5, 6]])

In [32]:
 x = x*x

In [33]:
x.decrypt(private_key = keys[0])

tensor([[ 1,  4,  9],
        [16, 25, 36]])